In [30]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io
from scipy.spatial.distance import cosine
import zipfile


In [ ]:
################################ HELPER FUNCTIONS ################################ 

# create condlist
conditionlist = {
    'leading_Barn': 0,
    'leading_beach': 1,
    'leading_library': 2,
    'leading_restaurant': 3, 
    'leading_cave': 4,
    'trailing_church': 5,
    'trailing_conference_room': 6,
    'trailing_castle': 7, 
    'trailing_forest': 8      
}

#Create category dict based on conditionlist

    #Idea: we create a customised event_id dict that allows us to later just extract 
    # the trailing images corresponding to our transitional probabiliy categories

    # Mapping VALID CONDITIONS (75%): 
        # leading_barn -> trailing_church
        # leading_beach -> trailing_church
        # leading_library -> trailing_conference_room
        # leading_restaurant ->  trailing_conference_room

    # Mapping INVALID CONDITIONS (25%): 
        # leading_barn -> trailing_conference_room
        # leading_beach -> trailing_conference_room
        # leading_library -> trailing_church
        # leading_restaurant -> trailing_church
        
    # Mapping CONTROL CONDITIONS (50%): 
        # leading_cave -> trailing_castle
        # leading_cave -> trailing_forest
    
category_dict = {
    0: (0, 5), # valid conditions 75 %
    1: (1, 5), # leading_beach -> trailing_church
    2: (2, 6), # leading_library -> trailing_conference_room
    3: (3, 6),  # leading_restaurant ->  trailing_conference_room
    4: (0, 6), # invalid conditions 0.25 %
    5: (1, 6),
    6: (2, 5), 
    7: (3, 5), 
    8: (4, 7), # control conditions 50 %
    9: (4, 8)
}


# Draw batch of images for testing

'''def batch_drawing_function(category_dict):
  # add the drwaing according to the mapping
  # 35 leading
  # 45 trailing '''





# Define resultspath
results_path_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/rdms/ann_RDM/pcn/"
results_path_bp = "/Users/denisekittelmann/Documents/Python/BiMoL/results/rdms/ann_RDM/backprop/"





In [ ]:
# Create a dict that assigns the correct labels for each leading-trailing imgage pair

"""
L1 = barn = label 0 - cat 1
L2 = beach = label 1 - cat 2
L5 = cave = label 2 - cat 3
L3 = library = label - cat 4
L4 = restaurant = label 4 - cat 5


L1 = barn = label 0 
L2 = beach = label 1
L3 = cave = label 2
L4 = library = label 3
L5 = restaurant = label 4 

    % Map 1 = C1 LEADING >> C6 TRAILING valid, C7 invalid
    % Map 2 = C2 LEADING >> C6 TRAILING valid, C7 invalid
    % Map 3 = C4 LEADING >> C7 TRAILING valid, C6 invalid
    % Map 4 = C5 LEADING >> C7 TRAILING valid, C6 invalid
    % Map 5 = C3 LEADING >> C8 OR C9 TRAILING
    % Map 6 = C3 LEADING >> C9 OR C9 TRAILING


T6 = Church = label 1   
T7 = conference room = label 2
T8 = castle = label 0   
T9 = forest = label 3

MAPPING:

L1 -> T6 = 0.75 -> (0,1) 
L1 -> T7 = 0.25 -> (0,2)
L1 -> T8 = 0 -> (0,0)
L1 -> T9 = 0 -> (0,3)

L2 -> T6 = 0.75 -> (1,1) 
L2 -> T7 = 0.25 -> (1,2)
L2 -> T8 = 0 -> (1,0)
L2 -> T9 = 0 -> (1,3)

L3 -> T6 = 0 -> (3,1) 
L3 -> T7 = 0 -> (3,2)
L3 -> T8 = 0.5 -> (3,0)
L3 -> T9 = 0.5 -> (3,3)

L4 -> T6 = 0.25 -> (4,1) 
L4 -> T7 = 0.75 -> (4,2)
L4 -> T8 = 0 -> (4,0)
L4 -> T9 = 0 -> (4,3)

L5 -> T6 = 0.25 -> (2,1) 
L5 -> T7 = 0.75 -> (2,2)
L5 -> T8 = 0 -> (2,0)
L5 -> T9 = 0 -> (2,3)

"""


label_dict = {
    (0, 1): 0.0,
    (0, 2): 0.75,
    (0, 0): 0.25,
    (0, 3): 0.25,
    
    (1, 1): 0.0,
    (1, 2): 0.75,
    (1, 0): 0.25,
    (1, 3): 0.25,
    
    (3, 1): 0.75,
    (3, 2): 0.75,
    (3, 0): 0.50,
    (3, 3): 0.50,
    
    (4, 1): 0.75,
    (4, 2): 0.0,
    (4, 0): 0.25,
    (4, 3): 0.25,
    
    (2, 1): 0.75, 
    (2, 2): 0.0,
    (2, 0): 0.25,
    (2, 3): 0.25
}


# Mapping VALID CONDITIONS (75%): 
    # leading_barn -> trailing_church
    # leading_beach -> trailing_church
    # leading_library -> trailing_conference_room
    # leading_restaurant ->  trailing_conference_room

# Mapping INVALID CONDITIONS (25%): 
    # leading_barn -> trailing_conference_room
    # leading_beach -> trailing_conference_room
    # leading_library -> trailing_church
    # leading_restaurant -> trailing_church
    
# Mapping CONTROL CONDITIONS (50%): 
    # leading_cave -> trailing_castle
    # leading_cave -> trailing_forest

L1 = barn = label 0 
L2 = beach = label 1
L3 = cave = label 2
L4 = library = label 3
L5 = restaurant = label 4 

0 -> 1
1 -> 1
3 -> 2
4 -> 2

0 -> 2 
1 -> 2
3 -> 1
4 -> 1

2 -> 0 
2 -> 3 







T6 = Church = label 1   
T7 = conference room = label 2
T8 = castle = label 0   
T9 = forest = label 3


category_dict = {
    0: (0, 1), # valid conditions 75 %
    1: (1, 1), # leading_beach -> trailing_church
    2: (3, 2), # leading_library -> trailing_conference_room
    3: (4, 2),  # leading_restaurant ->  trailing_conference_room
    4: (0, 2), # invalid conditions 0.25 %
    5: (1, 2),
    6: (3, 1), 
    7: (4, 1), 
    8: (2, 0), # control conditions 50 %
    9: (2, 3)
}


In [32]:
# Image paths 

img_dir_lead = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Leading/'
img_dir_trail = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Trailing/'
img_dir_test_lead = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Test/Test_Leading/'
img_dir_test_trail = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Test/Test_Trailing/'
class_names_L = ['barn', 'beach', 'cave', 'library', 'restaurant']
class_names_T = ['castle', 'Church', 'conference_room', 'forest'] # changed the order 
batch_size = None # adjust if needed, e.g., 32
image_size = (28,28)
validation_split = 0.1
seed = 123

In [42]:
def tf_scale_imgs(imgs, scale_factor):
    return imgs * scale_factor + 0.5 * (1 - scale_factor) * tf.ones(imgs.shape)


def tf_scale_labels(labels, scale_factor):
    return labels * scale_factor + 0.5 * (1 - scale_factor) * tf.ones(labels.shape)


def tf_f_inv(x, act_fn):
    """ (activation_size, batch_size) """
    if act_fn == "LINEAR":
        m = x
    elif act_fn == "TANH":
        num = tf.ones_like(x) + x
        div = tf.ones_like(x) - x + 1e-7
        m = 0.5 * tf.math.log(num / div)
    elif act_fn == "LOGSIG":
        div = tf.ones_like(x) - x + 1e-7
        m = tf.math.log((x / div) + 1e-7)
    else:
        raise ValueError(f"{act_fn} not supported")
    return m


def img_preproc(x, y, dtype=tf.float32): 
  """Cast input image to a certain tf dtype and normalize them between 0 and 1."""
  x = tf.cast(x, dtype) / 255.
  #x = tf_scale_imgs(x, cf.img_scale)
  #y = tf_scale_labels(y, cf.label_scale)
  #x = tf_f_inv(x, "TANH")
  #y = tf.one_hot(y, depth=10)
  return x, y


def flatten(x, y):  
  #flattens a video image series (or batch of images) to (n_batch, n_steps, 1) d.
  shape = tf.shape(x)
  if len(shape) == 5: # hack, determining if it's a video or not (batch_size, n_steps, height, width, channels)
    x = tf.reshape(x, [shape[0], shape[1], -1])
  elif len(shape) == 4: # regular image (batch_size, height, width, channels)
    x = tf.reshape(x, [shape[0], -1])
  elif len(shape) == 3:  # Single image (height, width, channels)
    x = tf.reshape(x, [-1])
  return x, y

def augment_images(batch_images, batch_labels):
    """
    Applies data augmentation on a batch of images without TensorFlow Addons.
    
    Parameters:
    batch_images: Tensor of shape (n_batch, 56, 28, 3)
    
    Returns:
    Augmented batch of images.
    """
    # Random horizontal flip
    augmented_images = tf.image.random_flip_left_right(batch_images)
    
    # Random brightness adjustment
    augmented_images = tf.image.random_brightness(augmented_images, max_delta=0.1)
    
    # Random contrast adjustment
    augmented_images = tf.image.random_contrast(augmented_images, lower=0.9, upper=1.1)
    
    # Random saturation adjustment
    augmented_images = tf.image.random_saturation(augmented_images, lower=0.9, upper=1.1)
    
    # Random hue adjustment
    augmented_images = tf.image.random_hue(augmented_images, max_delta=0.05)
    
    # Clipping to ensure pixel values are valid after transformations
    augmented_images = tf.clip_by_value(augmented_images, 0.0, 1.0)
    
    return augmented_images, batch_labels


In [43]:
# Generate image pairs 
def img_sequence(img_t1, img_t2, label_t1, label_t2, label_dict): 
    """This function stacks two images to construct an image pair and assigns a single label based on the label dictionary."""
    
    img_t1 = tf.cast(img_t1, dtype=tf.float32)
    img_t2 = tf.cast(img_t2, dtype=tf.float32)
    
    x = tf.concat([img_t1, img_t2], axis=0) 

    key_t1 = int(label_t1.numpy())
    key_t2 = int(label_t2.numpy())
 
    
    if (key_t1, key_t2) in label_dict:
        label = label_dict[(key_t1, key_t2)]
        #print(f"Label value found: {label}")
    else:
        print(f"Label pair {(key_t1, key_t2)} not found.")

    
    y = tf.cast(tf.random.uniform([]) < label, tf.float32)
    y = tf.expand_dims(y, axis=0)  
    
    return x,y


In [ ]:
import tensorflow as tf
import itertools

def generate_all_combinations(img_dirt1, img_dirt2, class_namest1, class_namest2, label_dict, image_size=None, seed=None): 
    # Load images from both directories without batching
    data_t1 = tf.keras.preprocessing.image_dataset_from_directory(
        img_dirt1, 
        label_mode='int',
        class_names=class_namest1,
        batch_size=None,
        color_mode='rgb',
        image_size=image_size,
        seed=seed
    )
    
    data_t2 = tf.keras.preprocessing.image_dataset_from_directory(
        img_dirt2, 
        label_mode='int',
        class_names=class_namest2,
        batch_size=None,
        color_mode='rgb',
        image_size=image_size,
        seed=seed
    )
    
    # Convert datasets to lists for pairing
    data_t1_list = list(data_t1)
    data_t2_list = list(data_t2)

    # Create all possible combinations
    for (img_t1, label_t1), (img_t2, label_t2) in itertools.product(data_t1_list, data_t2_list):
        # Generate image pair and label
        x, y = img_sequence(img_t1, img_t2, label_t1, label_t2, label_dict)
        yield x, y


In [44]:
def generate_dataset(img_dirt1, img_dirt2, class_namest1, class_namest2, label_dict, image_size = None, seed = None, shuffle = False):       
    
    
    data_t1 = tf.keras.preprocessing.image_dataset_from_directory(
        img_dirt1, 
        label_mode = 'int',
        class_names= class_namest1,
        batch_size = None,
        color_mode = 'rgb',
        image_size = image_size, 
        #shuffle = True, 
        seed = seed
        )

    data_t2 = tf.keras.preprocessing.image_dataset_from_directory(
        img_dirt2, 
        label_mode = 'int',
        class_names= class_namest2,
        batch_size = None,
        color_mode = 'rgb', 
        image_size = image_size, 
        #shuffle = True, 
        seed = seed
    )
    
    if shuffle:
        data_t1.shuffle(99999, seed = seed*2)
        data_t2.shuffle(99999, seed = seed*3)
    
    # iterate through shuffled leading and trailing datasets
    leading = iter(data_t1)
    trailing = iter(data_t2) 
              
    while True:
        try:
            # Retrieve single samples
            img_t1, label_t1 = next(leading)
            img_t2, label_t2 = next(trailing)

            # Generate x, y pairs for single samples
            x, y = img_sequence(img_t1, img_t2, label_t1, label_t2, label_dict) 
            yield x, y
            
        except StopIteration:
            # Break the loop if no more samples
            break
        
        

In [45]:
################################ PCN NETWORK OBJECT ################################ 

class CustomDense(tf.keras.layers.Dense):
    def call(self, inputs):
        """This works like a dense, except for the activation being called earlier."""
        # Apply the activation to the input first
        activated_input = self.activation(inputs)
        # Perform the matrix multiplication and add the bias
        output = tf.matmul(activated_input, self.kernel)
        if self.use_bias:
            output = output + self.bias
        return output


class PredictiveCodingNetwork(tf.keras.Sequential):
    def __init__(self, layers, vars, beta, **kwargs):
        """Initialize a PredictiveCodingNetwork"""
        super().__init__(layers, **kwargs)
        self.vars = tf.convert_to_tensor(vars, dtype=tf.float32)
        self.beta = beta

    def call_with_states(self, x):
        x_list = [x]
        for layer in self.layers:
            x = layer(x)
            x_list.append(x)
        return x_list

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        # do the stuff we do in train_epochs
        outputs, errors = self.infer(x, y)
        self.update_params(outputs, errors)

        # Update metrics (includes the metric that tracks the loss)
        pred = self.call(x)
        for metric in self.metrics:
            metric.update_state(y, pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
    
   
    def infer(self, x_batch, y_batch=None, n_iter=50, return_sequence=False):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        if return_sequence:
            errors_time = []
            states_time = []
        errors = [None for _ in range(len(self.layers))]
        f_x_arr = [None for _ in range(len(self.layers))]
        f_x_deriv_arr = [None for _ in range(len(self.layers))]
        shape = x_batch.shape
        batch_size = shape[0]

        for itr in range(n_iter):
            # if its the first itr, set x to the current forward call
            if itr == 0:
                x = self.call_with_states(x_batch)

                if y_batch is not None:
                  x[-1] = y_batch
            else:
                # update g and x only for consecutive iterations
                for l in range(1, len(self.layers)):
                    g = tf.multiply(tf.matmul(errors[l], self.layers[l].kernel, transpose_b=True), f_x_deriv_arr[l])
                    x[l] = x[l] + self.beta * (-errors[l-1] + g)

            # update f_x etc for every iteration
            for l in range(len(self.layers)):
                f_x = self.layers[l].activation(x[l])
                f_x_deriv_fn = self.get_activation_derivative(self.layers[l].activation)
                f_x_deriv = f_x_deriv_fn(x[l])
                f_x_arr[l] = f_x
                f_x_deriv_arr[l] = f_x_deriv
                errors[l] = (x[l + 1] - tf.matmul(f_x, self.layers[l].kernel) - self.layers[l].bias) / self.vars[l]
            
            if return_sequence:
                errors_time.append(errors)
                states_time.append(x)

        # return what we want to return
        if return_sequence:
            states_time = [tf.stack(tensors, axis=1) for tensors in zip(*states_time)]
            errors_time = [tf.stack(tensors, axis=1) for tensors in zip(*errors_time)]
            return states_time, errors_time
        else:
            return x, errors
    
    # We need to check if we actually need call here.
    # Now, call will give us the result of the network after the first inference step
    # If we want to have the results after the last inference step, we would need to change this
    #def call(self, inputs, training=False):
    #    """Call, but time distributed."""
    #    x, errors = self.infer(inputs, return_sequence=False)
    #    return x[-1]

    def update_params(self, x, errors):
        """Update the model parameters."""
        batch_size = tf.cast(tf.shape(x[0])[0], tf.float32)
        gradients = []
        for l, layer in enumerate(self.layers):
            grad_w = self.vars[-1] * (1 / batch_size) * tf.matmul(tf.transpose(self.layers[l].activation(x[l])), errors[l])
            grad_b = self.vars[-1] * (1 / batch_size) * tf.reduce_sum(errors[l], axis=0)
            gradients.append((-grad_w, layer.kernel))
            gradients.append((-grad_b, layer.bias))
        self.optimizer.apply_gradients(gradients)

    def get_activation_derivative(self, activation):
        """Return a function for the derivative of the given activation function."""
        activation_fn = tf.keras.activations.get(activation)
        if activation_fn == tf.keras.activations.linear:
            return lambda x: tf.ones_like(x)
        elif activation_fn == tf.keras.activations.tanh:
            return lambda x: 1 - tf.square(tf.nn.tanh(x))
        elif activation_fn == tf.keras.activations.sigmoid:
            return lambda x: tf.nn.sigmoid(x) * (1 - tf.nn.sigmoid(x))
        else:
            raise ValueError(f"{activation} not supported")
        

model = PredictiveCodingNetwork([CustomDense(units=6, activation="sigmoid"),
                                 CustomDense(units=4, activation="sigmoid"), 
                                 CustomDense(units=1, activation="sigmoid")], 
                                vars=[1, 1, 1], # variances. This is super useless and in the code only the last variance is used
                                beta=0.1)



In [66]:
seed = 123

val_dataset = tf.data.Dataset.from_generator(
    lambda: generate_dataset(img_dir_test_lead, img_dir_test_trail, class_names_L, class_names_T, label_dict, image_size = (28,28), seed = seed),
    output_signature=(
        tf.TensorSpec(shape=(56, 28, 3), dtype=tf.float32),  # shape of x 
        tf.TensorSpec(shape=(1), dtype=tf.float32)  # shape of y 
    )
) 

print(val_dataset)

<_FlatMapDataset element_spec=(TensorSpec(shape=(56, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>


In [74]:
# Step 1: Unzip the .keras file to access its internal structure
keras_file_path = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/model_checkpoint_pcnoriginal_726_0.67.keras"
unzip_path = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/pcn_tt/"

with zipfile.ZipFile(keras_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# Step 2: Instantiate your custom model with the correct parameters
correct_model = model # PCN loaded after reinitialising the network


# Now `correct_model` has the loaded weights
correct_model.build([None, 4704])
correct_model.load_weights("/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/pcn_tt/model.weights.h5")

correct_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-7, weight_decay=1e-2),
    loss="categorical_crossentropy",  # Placeholder loss
    metrics=["accuracy"]
)

# Predict states
predictions = correct_model.predict(val_dataset.batch(512).map(img_preproc).map(flatten))

# Evaluate the model
test_loss, test_accuracy = correct_model.evaluate(val_dataset.batch(512).map(img_preproc).map(flatten))
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


Found 180 files belonging to 5 classes.
Found 180 files belonging to 4 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
Found 180 files belonging to 5 classes.
Found 180 files belonging to 4 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 820ms/step - accuracy: 0.6722 - loss: 3.9074e-08
Test Loss: 3.907416257220575e-08, Test Accuracy: 0.6722221970558167


In [ ]:

################################ LOAD ANNs ################################ 

# In find_bestmodel.py we identified the best models for both networks: 
    # PCN: model at epoch 726 -> 75% 
    # Backprop_ann: model at epoch 649 -> 75% 
    
    
    
    
    
    
    
    
    

dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/updated_model_checkpoint.keras" #"/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/model_checkpoint_pcnoriginal_726_0.67.keras"
dir_bpann = "/Users/denisekittelmann/Documents/Python/BiMoL/results/bp_ann/model_checkpoint_649_0.67.keras"

# Load the models
#pcn = tf.keras.models.load_model(dir_pcn, custom_objects={"CustomDense": CustomDense, "PredictiveCodingNetwork": PredictiveCodingNetwork})
bp_ann = tf.keras.models.load_model(dir_bpann)



pcn = tf.keras.models.load_model(
    dir_pcn,
    custom_objects={
        "CustomDense": CustomDense,
        "PredictiveCodingNetwork": PredictiveCodingNetwork
    }
)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/denisekittelmann/miniconda3/envs/BiMo_3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/nn/03q5gpsn00zd2frzvtms7bh80000gn/T/ipykernel_35599/3913104978.py", line 16, in <module>
    pcn = tf.keras.models.load_model(
  File "/Users/denisekittelmann/miniconda3/envs/BiMo_3.9/lib/python3.9/site-packages/keras/src/saving/saving_api.py", line 187, in load_model
    return saving_lib.load_model(
  File "/Users/denisekittelmann/miniconda3/envs/BiMo_3.9/lib/python3.9/site-packages/keras/src/saving/saving_lib.py", line 365, in load_model
    return _load_model_from_fileobj(
  File "/Users/denisekittelmann/miniconda3/envs/BiMo_3.9/lib/python3.9/site-packages/keras/src/saving/saving_lib.py", line 442, in _load_model_from_fileobj
    model = _model_from_config(
  File "/Users/denisekittelmann/miniconda3/envs/BiMo_3.9/lib/python3.9/site-p

In [29]:
model = PredictiveCodingNetwork(vars=[1, 1, 1], beta=0.1)

# Load weights
dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/model_checkpoint_pcnoriginal_726_0.67.keras"
model.load_weights(dir_pcn)

# Save to a new file
updated_dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/updated_model_checkpoint.keras"
model.save(updated_dir_pcn)

AttributeError: 'TrackedList' object has no attribute 'numpy'

In [ ]:
# Step 1: Rebuild the Model Architecture
# Replace these with the actual parameters and layers you used originally
layers = [CustomDense(...), CustomDense(...)]  # Define your layers as in the original model
vars = [...]  # The vars used in the original model
beta = 0.5  # Replace with the original beta value

# Initialize the PredictiveCodingNetwork with the rebuilt architecture
pcn_rebuilt = PredictiveCodingNetwork(layers=layers, vars=vars, beta=beta)

# Step 2: Load Weights from the Existing Model File
# Specify the path to the original saved model file
dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/model_checkpoint_pcnoriginal_726_0.67.keras"
pcn_rebuilt.load_weights(dir_pcn)  # Loads the weights from the saved file

# Step 3: Save the Rebuilt Model
# Now save it to a new file, which includes the updated `get_config` and `from_config`
updated_dir_pcn = "/Users/denisekittelmann/Documents/Python/BiMoL/results/pcn/updated_model_checkpoint.keras"
pcn_rebuilt.save(updated_dir_pcn)


In [ ]:
################################ COMPUTE ANN RDMs ################################ 

# list all layer names
for layer in pcn.layers:
    print(layer.name)
    
for layer in bp_ann.layers:
    print(layer.name)    
    
# Extract Layer 1’s output
layer1_output = pcn.get_layer('layer_name_1').output # output layer 1
layer1_output = bp_ann.get_layer('layer_name_1').output# adapt 

layer1_model_pcn = Model(inputs=pcn.input, outputs=layer1_output) 


def batch_drawing_function(valid_cat):
  # add the drwaing according to the mapping
  # 35 leading
  # 45 trailing 
  

rdm_h1_pcn = np.zeros([9, 9])
rdm_h1_bp = np.zeros([9, 9])

rdm_h23_pcn_l1 = np.zeros([9, 9])
rdm_h23_pcn_l2 = np.zeros([9, 9])
rdm_h23_bp_l1 = np.zeros([9, 9])
rdm_h23_bp_l2 = np.zeros([9, 9])

#rdm_h2 = np.zeros([9, 9])
#rdm_h3 =np.zeros([9, 9])

for category_l in category_dict:
  for category_t in category_dict: 

    batch_i = batch_drawing_function(category_l)
    pcn_acts_i = pcn(batch_i)

    batch_j = batch_drawing_function(category_t)
    pcn_acts_j = pcn(batch_j)
    
    batch_i = batch_drawing_function(category_l)
    bp_acts_i = bp_ann(batch_i)

    batch_j = batch_drawing_function(category_t)
    bp_acts_j = bp_ann(batch_j)
    
    
    # H1
    rdm_h1_pcn[category_l, category_t] = cosine(np.mean(pcn_acts_i[0], axis=0), np.mean(pcn_acts_j[0], axis=0))
    rdm_h1_bp[category_l, category_t] = cosine(np.mean(bp_acts_i[1], axis=0), np.mean(bp_acts_j[1], axis=0))
    
    # H2 &H3
    rdm_h23_pcn_l1[category_l, category_t] = cosine(np.mean(np.concatenate([layer for layer in pcn_acts_i], axis=1), axis=0), np.mean(np.concatenate([layer for layer in pcn_acts_j], axis=1), axis=0))
    rdm_h23_bp_l1[category_l, category_t] = cosine(np.mean(np.concatenate([layer for layer in bp_acts_i], axis=1), axis=0), np.mean(np.concatenate([layer for layer in bp_acts_j], axis=1), axis=0)) 
    
    rdm_h23_pcn_l2[category_l, category_t] = cosine(np.mean(np.concatenate([layer for layer in pcn_acts_i], axis=1), axis=0), np.mean(np.concatenate([layer for layer in pcn_acts_j], axis=1), axis=0))
    rdm_h23_bp_l2[category_l, category_t] = cosine(np.mean(np.concatenate([layer for layer in bp_acts_i], axis=1), axis=0), np.mean(np.concatenate([layer for layer in bp_acts_j], axis=1), axis=0)) 
    

   # Save the RDMs 
    #np.save(os.path.join(results_path_pcn, "pcn_rdm_h1.npy"), rdm_h1_pcn)
    #np.save(os.path.join(results_path_pcn, "pcn_rdm_h23_l1.npy"), rdm_h23_pcn_l1)
    #np.save(os.path.join(results_path_pcn, "pcn_rdm_h23_l1.npy"), rdm_h23_pcn_l2)
    #np.save(os.path.join(results_path_bp, "bp_rdm_h1.npy"), rdm_h1_bp)
    #np.save(os.path.join(results_path_bp, "bp_rdm_h23_l1.npy"), rdm_h23_bp_l1)
    #np.save(os.path.join(results_path_bp, "bp_rdm_h23_l2.npy"), rdm_h23_bp_l2)
    
    
    

In [ ]:
plt.imshow(rdm)
plt.colorbar()